In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
import google.auth.transport.requests 
from google.auth.transport.requests import AuthorizedSession
import gspread
from datetime import datetime


In [34]:
now = datetime.now()
#now_bogota = now.astimezone(timezone('America/Sao_Paulo'))
today = now.strftime("%d/%m/%Y")

cid = open(r"G:\Meu Drive\MBA\DAS 0013\spotify_confidential.txt.txt", "r").read().split('\n')[0]
secret=open(r"G:\Meu Drive\MBA\DAS 0013\spotify_confidential.txt.txt", "r").read().split('\n')[1]
key = '18MFV_mk8PAem4aaO0Psl6SMTmH6TCsdLNZGJvLsGNVQ'


In [35]:
def pandas_df_to_googlesheets_(key,Data,path="G:\Meu Drive\MBA\DAS 0013\sheets-credentials.json"):
    Dataset = Data.copy().astype(str)
    Valores = Dataset.values
    Columnas = Dataset.columns.values

    scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']; #Apis autorizadas
    credentials = Credentials.from_service_account_file(path,scopes=scopes); #Se lee lacredencial
    #requests.Session()
    request = google.auth.transport.requests.Request()
   
    credentials.refresh(request)

    gc = gspread.authorize(credentials)
    sht = gc.open_by_key(str(key)); #Se lee la tabla de calculo correspondiente

    worksheet = sht.worksheet('RawData'); #Se extrae la hoja   

    response = worksheet.get_values('A1:A')

    last_row = str(len(response)+1)

    worksheet.update("A" + last_row,Valores.tolist())

In [36]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [37]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [38]:
lst_data = []

In [39]:
#df = pd.DataFrame(columns =['track_uri','track_name','artist_uri','artist_info','artist_name','artist_pop','artist_genres','album','track_pop'])
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    """
    data = [
        ['track_uri', track_uri],
        ['track_name', track_name],
        ['artist_uri', artist_uri],
        ['artist_info', artist_info],
        ['artist_name', artist_name],
        ['artist_pop', artist_pop],
        ['artist_genres', artist_genres],
        ['album', album],
        ['track_pop', track_pop]
        ]
    """
    data = [
        today,
        track_uri,
        track_name,
        artist_uri,
        artist_info,
        artist_name,
        artist_pop,
        artist_genres,
        album,
        track_pop
        ]
    
    #print(len(data))
    #df.loc[-1] = [data]  # adding a row
    #df.index = df.index + 1  # shifting index
    df1 = pd.DataFrame([data],columns=['date','track_uri','track_name','artist_uri','artist_info','artist_name','artist_pop','artist_genres','album','track_pop'])
    lst_data.append(df1)

df = pd.concat(lst_data, ignore_index=True)


In [40]:
df.head()

,date,track_uri,track_name,artist_uri,artist_info,artist_name,artist_pop,artist_genres,album,track_pop
0,29/09/2022,spotify:track:7gRFDGEzF9UkBV233yv2dc,Shut Down,spotify:artist:41MozSoPIsD1dJM0CLPjZF,{'external_urls': {'spotify': 'https://open.sp...,BLACKPINK,89,"[k-pop, k-pop girl group]",BORN PINK,77
1,29/09/2022,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,"Quevedo: Bzrp Music Sessions, Vol. 52",spotify:artist:716NhGYqD1jl2wI1Qkgq36,{'external_urls': {'spotify': 'https://open.sp...,Bizarrap,86,"[argentine hip hop, pop venezolano, trap argen...","Quevedo: Bzrp Music Sessions, Vol. 52",100
2,29/09/2022,spotify:track:4Dvkj6JhhA12EX05fT7y2e,As It Was,spotify:artist:6KImCVD70vtIoJWnq6nGn3,{'external_urls': {'spotify': 'https://open.sp...,Harry Styles,91,[pop],Harry's House,93
3,29/09/2022,spotify:track:4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),spotify:artist:1Cs0zKBU1kc0i8ypK3B9ai,{'external_urls': {'spotify': 'https://open.sp...,David Guetta,88,"[big room, dance pop, edm, pop, pop dance]",I'm Good (Blue),96
4,29/09/2022,spotify:track:5zwwW9Oq7ubSxoCGyW1nbY,Pink Venom,spotify:artist:41MozSoPIsD1dJM0CLPjZF,{'external_urls': {'spotify': 'https://open.sp...,BLACKPINK,89,"[k-pop, k-pop girl group]",BORN PINK,75


In [41]:
pandas_df_to_googlesheets_(key,df)